# Study how the batch size affects the performance across installed Caffe variants and models

## Includes

### Standard

In [ ]:
import os
import sys
import json
import re
from string import Template

### Scientific

In [ ]:
import IPython as ip
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mp

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('NumPy version: %s' % np.__version__)
print ('SciPy version: %s' % sp.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('Matplotlib version: %s' % mp.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
# import scipy.stats as st

### Collective Knowledge

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

## Access Caffe installations

In [ ]:
caffe_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'lib,caffe'})
caffe_lst = caffe_env['lst']
for (caffe, caffe_index) in zip(caffe_lst, range(len(caffe_lst))):
    # Get the tags from e.g. 'BVLC Caffe framework (clblast)'
    caffe_tags = re.match('BVLC Caffe framework \((?P<tags>.*)\)', caffe['info']['data_name'])
    caffe_tags = caffe_tags.group('tags').replace(', ','-')
    print ("%d - %s - %s" % (caffe_index, caffe_tags, caffe['data_uid']))

## Access Caffe models

In [ ]:
caffemodel_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'caffemodel'})
caffemodel_lst = caffemodel_env['lst']
for (caffemodel, caffemodel_index) in zip(caffemodel_lst, range(len(caffemodel_lst))):
    # Get the tags from e.g. 'Caffe model (net and weights) (deepscale, squeezenet, 1.1)'
    caffemodel_tags = re.match('Caffe model \(net and weights\) \((?P<tags>.*)\)', caffemodel['info']['data_name'])
    caffemodel_tags = caffemodel_tags.group('tags').replace(', ','-')
    print ("%d - %s - %s" % (caffemodel_index, caffemodel_tags, caffemodel['data_uid']))

## Generate all the benchmarking files

In [ ]:
setup_program_pipeline_json_template = Template('''\
{
    "prepare":"yes",

    "save_to_file":"$setup_program_pipeline_tmp_json_path",

    "data_uoa":"caffe",

    "cmd_key":"time_gpu",

    "speed":"yes",
    "energy":"no",

    "cpu_freq":"max",
    "gpu_freq":"max",

    "no_compiler_description":"yes",
    "no_state_check":"yes",
    "compile_only_once":"yes",
    "skip_calibration":"yes"
}
''')

In [ ]:
explore_batch_size_json_template = Template('''\
{
    "choices_order":[
        [
            "##env#CK_CAFFE_BATCH_SIZE"
        ]
    ],
    "choices_selection":[
        {"type":"loop", "start":$start, "stop":$stop, "step":$step, "default":$default}
    ],

    "features_keys_to_process":["##choices#*"],

    "iterations":-1,
    "repetitions":5,

    "record":"yes",
    "record_failed":"yes",
    "record_params":{
        "search_point_by_features":"yes"
    },
    "record_repo":"local",
    "record_uoa":"$record_uoa",

    "tags":["time_gpu", "$hostname", "$caffemodel_tags", "$caffe_tags"]
}
''')

In [ ]:
explore_batch_size_bash_template = Template('''\
#!/bin/bash
ck autotune pipeline:program pipeline_from_file=$setup_program_pipeline_tmp_json_path @$explore_batch_size_json_path
''')

In [ ]:
def generate(caffe_tags, caffemodel_tags, caffe_index, caffemodel_index):
    # Generate the JSON files for setting up the program pipeline.
    setup_program_pipeline_json_path = 'pipeline.%s.%s.json' % (caffemodel_tags, caffe_tags)
    setup_program_pipeline_tmp_json_path = 'pipeline.%s.%s.tmp.json' % (caffemodel_tags, caffe_tags)
    with open(setup_program_pipeline_json_path, 'w') as setup_program_pipeline_json_file:
        setup_program_pipeline_json_file.write(
            setup_program_pipeline_json_template.substitute(
                setup_program_pipeline_tmp_json_path=setup_program_pipeline_tmp_json_path
            )
        )

    # Generate answers for when setting up the program pipeline.
    setup_program_pipeline_answers_path = 'pipeline.%s.%s.answers' % (caffemodel_tags, caffe_tags)
    !echo -n > {setup_program_pipeline_answers_path}
    !echo {caffe_index}      >> {setup_program_pipeline_answers_path}
    !echo {caffemodel_index} >> {setup_program_pipeline_answers_path}
    
    # Clean and then set up the program pipeline. NB: IPython requires double braces for the argument to echo.
    !echo {{}} > {setup_program_pipeline_tmp_json_path}
    !cat {setup_program_pipeline_answers_path} | ck pipeline program @{setup_program_pipeline_json_path} @{setup_program_pipeline_tmp_json_path} "$@"
    
    # Generate the JSON file for exploring the batch size.
    hostname = !hostname
    choices_selection = { 'start':1, 'stop':4, 'step':1, 'default':1 }
    explore_batch_size_json_path = 'explore.%s.%s.json' % (caffemodel_tags, caffe_tags)
    with open(explore_batch_size_json_path, 'w') as explore_batch_size_json_file:
        explore_batch_size_json_file.write(
            explore_batch_size_json_template.substitute(
                record_uoa='%s-%s' % (caffemodel_tags, caffe_tags),
                caffemodel_tags=caffemodel_tags,
                caffe_tags=caffe_tags,
                hostname=hostname.s,
                start=choices_selection['start'],
                stop=choices_selection['stop'],
                step=choices_selection['step'],
                default=choices_selection['default']
            )
        )
    
    # Generate the bash file for exploring the batch size.
    explore_batch_size_bash_path = 'explore.%s.%s.sh' % (caffemodel_tags, caffe_tags)
    with open(explore_batch_size_bash_path, 'w') as explore_batch_size_bash_file:
        explore_batch_size_bash_file.write(
            explore_batch_size_bash_template.substitute(
                setup_program_pipeline_tmp_json_path=setup_program_pipeline_tmp_json_path,
                explore_batch_size_json_path=explore_batch_size_json_path
            )
        )
    !chmod u+x {explore_batch_size_bash_path}
    
    return os.path.join(os.path.curdir, explore_batch_size_bash_path)

In [ ]:
explore_batch_size_all_bash_path = 'explore.all.sh'
!echo "#!/bin/bash" > {explore_batch_size_all_bash_path}

In [ ]:
# All Caffe installations.
caffe_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'lib,caffe'})
caffe_lst = caffe_env['lst']

# All Caffe models.
caffemodel_env = ck.access({'action':'show', 'module_uoa':'env', 'tags':'caffemodel'})
caffemodel_lst = caffemodel_env['lst']

# For each Caffe installation.
for (caffe, caffe_index) in zip(caffe_lst, range(len(caffe_lst))):
    caffe_tags = re.match('BVLC Caffe framework \((?P<tags>.*)\)', caffe['info']['data_name'])
    caffe_tags = caffe_tags.group('tags').replace(', ','-')
    # For each Caffe model.
    for (caffemodel, caffemodel_index) in zip(caffemodel_lst, range(len(caffemodel_lst))):
        caffemodel_tags = re.match('Caffe model \(net and weights\) \((?P<tags>.*)\)', caffemodel['info']['data_name'])
        caffemodel_tags = caffemodel_tags.group('tags').replace(', ','-')
        explore_batch_size_bash_path = generate(caffe_tags, caffemodel_tags, caffe_index, caffemodel_index)
        !echo {explore_batch_size_bash_path} >> {explore_batch_size_all_bash_path}

!chmod u+x {explore_batch_size_all_bash_path}